## **1. Install and import bibraries**


In [1]:
# !pip install datasets evaluate accelerate
# !pip install causal-conv1d>=1.1.0
# !pip install mamba-ssm
# !pip install wandb # Nếu sử dụng wandb để log quá trình train

!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

from IPython.display import clear_output
clear_output()

In [2]:
import os
import random
import json
import torch
import torch.nn as nn
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import Trainer
from transformers import AutoTokenizer, TrainingArguments

/home/yenling/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Login into huggingface_hub to push trained model

In [3]:
# from huggingface_hub import notebook_login
# notebook_login()

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## **2. Download dataset**


In [5]:
# Tải bộ dataset
imdb = load_dataset("imdb")

## **3. EDA dataset**


There are two fields in this dataset:

* text: the movie review text.
* label: a value that is either 0 for a negative review or 1 for a positive review.

In [6]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [7]:
type(imdb["test"][0]['label'])

int

## **4. Build Custom Mamba Model for Text Classification**


In [8]:
# Config class của Mamba tham khảo từ: https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/config_mamba.py
@dataclass
class MambaConfig:
    d_model: int = 2560
    n_layer: int = 64
    vocab_size: int = 50277
    ssm_cfg: dict = field(default_factory=dict)
    rms_norm: bool = True
    residual_in_fp32: bool = True
    fused_add_norm: bool = True
    pad_vocab_size_multiple: int = 8
    tie_embeddings: bool = True

    def to_json_string(self):
        return json.dumps(asdict(self))

    def to_dict(self):
        return asdict(self)

In [9]:
# Định nghĩa lớp head để phân loại
class MambaClassificationHead(nn.Module):
    def __init__(self, d_model, num_classes, **kwargs):
        super(MambaClassificationHead, self).__init__()
        # Sử dụng một lớp tuyến tính để thực hiện phân loại dựa trên đầu vào có kích thước d_model và num_classes cần phân loại.
        self.classification_head = nn.Linear(d_model, num_classes, **kwargs)

    def forward(self, hidden_states):
        return self.classification_head(hidden_states)

In [10]:
class MambaTextClassification(MambaLMHeadModel):
    def __init__(
        self,
        config: MambaConfig,
        initializer_cfg=None,
        device=None,
        dtype=None,
    ) -> None:
        super().__init__(config, initializer_cfg, device, dtype)

        # Tạo một đầu phân loại sử dụng MambaClassificationHead với kích thước đầu vào là d_model và số lớp là 2.
        self.classification_head = MambaClassificationHead(d_model=config.d_model, num_classes=2)

        del self.lm_head

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Truyền input_ids qua mô hình gốc để nhận hidden_states.
        hidden_states = self.backbone(input_ids)

        # Lấy trung bình của hidden_states theo chiều thứ 2 để tạo ra [CLS] feature đại điện
        mean_hidden_states = hidden_states.mean(dim=1)

        # Đưa mean_hidden_states qua đầu phân loại để nhận logits.
        logits = self.classification_head(mean_hidden_states)

        if labels is None:
          ClassificationOutput = namedtuple("ClassificationOutput", ["logits"])
          return ClassificationOutput(logits=logits)
        else:
          ClassificationOutput = namedtuple("ClassificationOutput", ["loss", "logits"])

          # Sử dụng hàm mất mát CrossEntropyLoss để tính loss.
          loss_fct = nn.CrossEntropyLoss()
          loss = loss_fct(logits, labels)

          return ClassificationOutput(loss=loss, logits=logits)

    def predict(self, text, tokenizer, id2label=None):
        input_ids = torch.tensor(tokenizer(text)['input_ids'], device=device)[None] # device = 'cuda'
        with torch.no_grad():
          logits = self.forward(input_ids).logits[0]
          label = np.argmax(logits.cpu().numpy())

        if id2label is not None:
          return id2label[label]
        else:
          return label

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        # Tải cấu hình từ mô hình đã được huấn luyện trước đó.
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)

        # Khởi tạo mô hình từ cấu hình và chuyển nó đến thiết bị và kiểu dữ liệu mong muốn.
        model = cls(config, device=device, dtype=dtype, **kwargs)

        # Tải trạng thái mô hình đã được huấn luyện trước đó.
        model_state_dict = load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype)
        model.load_state_dict(model_state_dict, strict=False)

        # In ra các tham số embedding mới được khởi tạo.
        print("Newly initialized embedding:", set(model.state_dict().keys()) - set(model_state_dict.keys()))
        return model

In [11]:
# Tải mô hình Mamba từ mô hình đã được huấn luyện trước đó.
model = MambaTextClassification.from_pretrained("state-spaces/mamba-130m")
model.to(device)

# Tải tokenizer của mô hình Mamba từ mô hình gpt-neox-20b.
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# Đặt id của token pad bằng id của token eos trong tokenizer.
tokenizer.pad_token_id = tokenizer.eos_token_id

Newly initialized embedding: {'classification_head.classification_head.weight', 'classification_head.classification_head.bias'}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
model

MambaTextClassification(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (classification_head): MambaClassificationHead(
    (classification_head): Linear(in_features=768, out_features=2, bias=True)
  )
)

## **5. Preprocess dataset**


In [13]:
# Tạo chức năng tiền xử lý để mã hóa văn bản và cắt bớt các chuỗi không dài hơn độ dài đầu vào tối đa của mã thông báo
def preprocess_function(examples):
    samples = tokenizer(examples["text"], truncation=True)
    # Không cần attention_mask
    # Cụ thể hơn về token masking của mamba có thể tham khảo: https://github.com/state-spaces/mamba/issues/49
    samples.pop('attention_mask')
    return samples

In [14]:
# Thực hiện mã hóa văn bản
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map: 100%|██████████| 25000/25000 [00:06<00:00, 3915.55 examples/s]


In [15]:
# Set seed cho hàm random
random.seed(42)

# Tạo tập train và test
train_dataset = tokenized_imdb["train"]
test_dataset = tokenized_imdb["test"]

# Tạo tập evaluation để đánh giá trong lúc train
# Do số lượng tập test lớn nên chỉ lấy mẫu 1% tập dữ liệu test để đánh giá
total_samples = len(test_dataset)
eval_samples = int(0.01 * total_samples)
eval_indices = random.sample(range(total_samples), eval_samples)
eval_dataset = test_dataset.select(eval_indices)

In [16]:
print('total_samples',total_samples)
print('eval_samples',eval_samples)
print('eval_indices',eval_indices)
print('eval_dataset',eval_dataset)

total_samples 25000
eval_samples 250
eval_indices [20952, 3648, 819, 24299, 9012, 8024, 7314, 4572, 24132, 3358, 22174, 24270, 17870, 2848, 19349, 13825, 1041, 976, 3070, 7164, 7623, 16559, 19726, 869, 18390, 6515, 23462, 21295, 22981, 17856, 13746, 7223, 14719, 19309, 9115, 212, 24864, 5231, 22876, 13848, 11149, 9105, 5094, 7055, 11029, 3349, 3039, 12449, 3169, 11763, 11270, 19782, 8667, 1423, 23911, 15054, 17571, 4090, 12403, 2582, 18089, 9606, 20599, 20267, 11850, 18918, 6300, 23087, 2279, 1501, 21668, 7467, 9482, 2614, 7628, 3309, 12455, 9108, 14857, 20830, 11954, 5329, 12130, 11641, 6865, 21960, 8748, 22997, 22398, 21234, 2339, 19960, 20806, 5607, 17502, 23892, 8021, 5354, 15147, 12433, 8845, 20971, 22549, 18250, 7196, 22433, 10626, 1832, 7505, 1051, 10336, 13145, 8773, 2168, 6913, 18585, 23524, 10311, 6967, 21477, 16358, 12964, 21064, 15035, 4681, 8679, 4575, 8081, 24411, 18394, 17661, 8609, 24478, 19155, 14038, 19121, 13087, 11861, 7186, 4532, 16696, 16171, 2978, 24765, 1543, 35

## **6. Evaluation metric**


In [17]:
# Tải module "accuracy" từ thư viện evaluate.
accuracy = evaluate.load("accuracy")

# Định nghĩa hàm compute_metrics để tính các độ đo hiệu suất (metrics) cho việc đánh giá mô hình.
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Lấy chỉ số của lớp có xác suất cao nhất trong predictions.
    predictions = np.argmax(predictions, axis=1)

    # Sử dụng module "accuracy" để tính độ chính xác dựa trên predictions và labels.
    return accuracy.compute(predictions=predictions, references=labels)

## **7. Trainning**


In [18]:
# Định nghĩa một lớp con MambaTrainer kế thừa từ lớp Trainer.
class MambaTrainer(Trainer):

    # Định nghĩa hàm compute_loss để tính toán hàm mất mát trong quá trình huấn luyện.
    def compute_loss(self, model, inputs, return_outputs=False):
        # 從輸入中取得 input_ids 和 labels 值。
        # print('inputs:', inputs)
        input_ids = inputs.pop("input_ids")
        print('input_ids:',input_ids.shape)
        labels = inputs.pop('labels')
        print('labels:',labels.shape)
        attentionMask = inputs.pop('attention_mask')
        print('attentionMask:',attentionMask.shape)
        # 使用 input_ids 和 labels 呼叫模型的前向函數來取得結果。
        outputs = model(input_ids=input_ids, labels=labels)

        # 從模型結果中取得損失值。
        loss = outputs.loss

        # 如果 return_outputs 為 True，則傳回損失和輸出，否則僅回傳損失。
        return (loss, outputs) if return_outputs else loss

    # Định nghĩa hàm save_model để lưu model trong quá trình huấn luyện.
    def save_model(self, output_dir = None, _internal_call = False):
        # Kiểm tra nếu thư mục lưu trữ không được chỉ định, sử dụng thư mục mặc định từ đối số 'args'.
        if output_dir is None:
            output_dir = self.args.output_dir

        # Nếu thư mục đầu ra không tồn tại, tạo mới nó.
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Lưu trạng thái của mô hình PyTorch vào file 'pytorch_model.bin' trong thư mục đầu ra.
        torch.save(self.model.state_dict(), f"{output_dir}/pytorch_model.bin")

        # Lưu trạng thái của tokenizer vào thư mục đầu ra.
        self.tokenizer.save_pretrained(output_dir)

        # Lưu cấu hình của mô hình vào file 'config.json' trong thư mục đầu ra.
        with open(f'{output_dir}/config.json', 'w') as f:
            json.dump(self.model.config.to_dict(), f)

In [19]:
# Định nghĩa tên project để log thông tin quá trình huấn luyện trên wandb
# os.environ["WANDB_PROJECT"] = "mamba_tutorial"

# Định nghĩa các tham số huấn luyện trong đối tượng TrainingArguments.
# Cụ thể hơn về các tham số hỗ trợ có thể tham khảo: https://huggingface.co/docs/transformers/main_classes/trainer
training_args = TrainingArguments(
    output_dir="mamba_text_classification", # Thư mục lưu trữ kết quả huấn luyện
    learning_rate=5e-5, # Tốc độ học
    per_device_train_batch_size=64, # Số lượng mẫu huấn luyện trên mỗi thiết bị
    per_device_eval_batch_size=16, # Số lượng mẫu đánh giá trên mỗi thiết bị
    num_train_epochs=1, # Số epoch huấn luyện
    warmup_ratio=0.01, # Tỉ lệ tăng dần tốc độ học trong giai đoạn warmup
    lr_scheduler_type="cosine", # Loại lịch trình tốc độ học (cosine là một lựa chọn phổ biến)
    report_to="none", # wandb nếu muốn đưa thông tin huấn luyện lên WandB để theo dõi quá trình
    evaluation_strategy="steps", # Xác định chiến lược đánh giá sau mỗi số bước (steps)
    eval_steps=0.1, # Số bước giữa các đợt đánh giá
    save_strategy="steps", # Xác định khi nào lưu trạng thái của mô hình (steps)
    save_steps=0.1, # Số bước giữa các lần lưu trạng thái của mô hình
    logging_strategy="steps", # Xác định khi nào in thông tin log (steps)
    logging_steps=1, # Số bước giữa các lần in thông tin log
    push_to_hub=False,
    # push_to_hub=True,  # Đẩy kết quả huấn luyện lên một nơi chia sẻ trực tuyến (Hub)
    load_best_model_at_end=True, # Load model có kết quả evaluation tốt nhất trong quá trình train
)

In [20]:
# Khởi tạo đối tượng MambaTrainer để thực hiện quá trình huấn luyện của mô hình.
trainer = MambaTrainer(
    model=model, # Mô hình cần huấn luyện
    train_dataset=train_dataset, # Dữ liệu huấn luyện
    eval_dataset=eval_dataset, # Dữ liệu đánh giá
    tokenizer=tokenizer, # Tokenizer sử dụng để mã hóa dữ liệu
    args=training_args, # Các tham số huấn luyện đã được định nghĩa trước đó
    compute_metrics=compute_metrics # Hàm tính các độ đo hiệu suất (metrics) cho đánh giá
    )

In [21]:
# Bắt đầu quá trình huấn luyện bằng cách gọi hàm train() trên đối tượng trainer.
trainer.train()

input_ids: torch.Size([64, 1208])
labels: torch.Size([64])
attentionMask: torch.Size([64, 1208])


OutOfMemoryError: CUDA out of memory. Tried to allocate 454.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Process 908699 has 17179869184.00 GiB memory in use. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 36.10 GiB is allocated by PyTorch, and 99.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)